In [1]:
# Below is my basic implementation of AES/Rijndael encryptction and decryption
# Numerous sources were used here, will be linked below
# https://cryptohack.org/challenges/aes/
# https://www.samiam.org/mix-column.html
# https://en.wikipedia.org/wiki/Rijndael_MixColumns
# https://en.wikipedia.org/wiki/Rijndael_S-box
# https://en.wikipedia.org/wiki/Finite_field_arithmetic#Rijndael's_(AES)_finite_field
# https://en.wikipedia.org/wiki/Advanced_Encryption_Standard#High-level_description_of_the_algorithm
# https://github.com/francisrstokes/AES-C/tree/main/src

In [13]:
# This implmentationan of multiplication on the Rijndael field x^8 + x^4 + x^3 + x + 1
# Is based on the wikipedia article https://en.wikipedia.org/wiki/Finite_field_arithmetic#Rijndael's_(AES)_finite_field

from itertools import chain as Unlist

def RijndealMultiply(a: int, b: int):
    m = 0b100011011
    assert ( { a, b, m } & {*range(0xFFFF)} ) == { a, b, m }
    
    A = [0] * 8
    B = [0] * 8
    M = [0] * 16
    Z = [0] * 16
    
    for n in range(8):
        A[n] = 0b1 & ( a >> n )
        B[n] = 0b1 & ( b >> n )
    for n in range(16):
        M[n] = 0b1 & ( m >> n )
    assert { *A, *B, *M, *Z } <= {0b1, 0b0} 
        
    
    for a_idx in range(8):
        for b_idx in range(8):
            bit = A[a_idx] & B[b_idx]
            assert bit in {0b1, 0b0}
            Z[ a_idx + b_idx ] ^= bit
            
    Degree = lambda polynomial: max( exp for exp, val in enumerate(polynomial) if val ) if 0b1 in polynomial else 0              
    degree_z = Degree(Z)
    degree_m = Degree(M)
    assert degree_m == 8
    assert degree_z > degree_m
    
    diff = -1
    
    while ( diff := Degree(Z) - Degree(M) ) >= 0:
        Mshift = [0] * diff + M
        for exp in range(1+Degree(Z)):
            Z[exp] ^= Mshift[exp]
    
    return sum( val * 2**exp for exp, val in enumerate(Z) )

# 0x53 and 0xCA are inverses so answer should be 1
RijndealMultiply(23, 47)

11

In [12]:
# Reimplementing Galois Multiplation from here https://github.com/francisrstokes/AES-C/blob/main/src/aes.c
# I did my own from scratch above as RijndaelMultiply, 
# but I'm going to use this here as I expect it is more performant and stable than mine

def GaloisMult(a, b):
    assert type(a) is int and type(b) is int
    assert 0 <= a <= 0xFF and 0 <= b <= 0xFF
    
    result = 0
    for i in range(8):
        if b & 0b1:
            result ^= a
        shiftEscapesField = a & 0x80
        a <<= 1
        if shiftEscapesField:
            a ^= 0x11B
        b >>= 1
    
    return result

GaloisMult(23, 47)

11

In [4]:
Sbox = [[ 0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76 ],
        [ 0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0 ],
        [ 0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15 ],
        [ 0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75 ],
        [ 0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84 ],
        [ 0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf ],
        [ 0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8 ],
        [ 0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2 ],
        [ 0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73 ],
        [ 0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb ],
        [ 0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79 ],
        [ 0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08 ],
        [ 0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a ],
        [ 0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e ],
        [ 0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf ],
        [ 0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16 ]]

InverseSbox =  [[ 0x52, 0x09, 0x6A, 0xD5, 0x30, 0x36, 0xA5, 0x38, 0xBF, 0x40, 0xA3, 0x9E, 0x81, 0xF3, 0xD7, 0xFB ],
                [ 0x7C, 0xE3, 0x39, 0x82, 0x9B, 0x2F, 0xFF, 0x87, 0x34, 0x8E, 0x43, 0x44, 0xC4, 0xDE, 0xE9, 0xCB ],
                [ 0x54, 0x7B, 0x94, 0x32, 0xA6, 0xC2, 0x23, 0x3D, 0xEE, 0x4C, 0x95, 0x0B, 0x42, 0xFA, 0xC3, 0x4E ],
                [ 0x08, 0x2E, 0xA1, 0x66, 0x28, 0xD9, 0x24, 0xB2, 0x76, 0x5B, 0xA2, 0x49, 0x6D, 0x8B, 0xD1, 0x25 ],
                [ 0x72, 0xF8, 0xF6, 0x64, 0x86, 0x68, 0x98, 0x16, 0xD4, 0xA4, 0x5C, 0xCC, 0x5D, 0x65, 0xB6, 0x92 ],
                [ 0x6C, 0x70, 0x48, 0x50, 0xFD, 0xED, 0xB9, 0xDA, 0x5E, 0x15, 0x46, 0x57, 0xA7, 0x8D, 0x9D, 0x84 ],
                [ 0x90, 0xD8, 0xAB, 0x00, 0x8C, 0xBC, 0xD3, 0x0A, 0xF7, 0xE4, 0x58, 0x05, 0xB8, 0xB3, 0x45, 0x06 ],
                [ 0xD0, 0x2C, 0x1E, 0x8F, 0xCA, 0x3F, 0x0F, 0x02, 0xC1, 0xAF, 0xBD, 0x03, 0x01, 0x13, 0x8A, 0x6B ],
                [ 0x3A, 0x91, 0x11, 0x41, 0x4F, 0x67, 0xDC, 0xEA, 0x97, 0xF2, 0xCF, 0xCE, 0xF0, 0xB4, 0xE6, 0x73 ],
                [ 0x96, 0xAC, 0x74, 0x22, 0xE7, 0xAD, 0x35, 0x85, 0xE2, 0xF9, 0x37, 0xE8, 0x1C, 0x75, 0xDF, 0x6E ],
                [ 0x47, 0xF1, 0x1A, 0x71, 0x1D, 0x29, 0xC5, 0x89, 0x6F, 0xB7, 0x62, 0x0E, 0xAA, 0x18, 0xBE, 0x1B ],
                [ 0xFC, 0x56, 0x3E, 0x4B, 0xC6, 0xD2, 0x79, 0x20, 0x9A, 0xDB, 0xC0, 0xFE, 0x78, 0xCD, 0x5A, 0xF4 ],
                [ 0x1F, 0xDD, 0xA8, 0x33, 0x88, 0x07, 0xC7, 0x31, 0xB1, 0x12, 0x10, 0x59, 0x27, 0x80, 0xEC, 0x5F ],
                [ 0x60, 0x51, 0x7F, 0xA9, 0x19, 0xB5, 0x4A, 0x0D, 0x2D, 0xE5, 0x7A, 0x9F, 0x93, 0xC9, 0x9C, 0xEF ],
                [ 0xA0, 0xE0, 0x3B, 0x4D, 0xAE, 0x2A, 0xF5, 0xB0, 0xC8, 0xEB, 0xBB, 0x3C, 0x83, 0x53, 0x99, 0x61 ],
                [ 0x17, 0x2B, 0x04, 0x7E, 0xBA, 0x77, 0xD6, 0x26, 0xE1, 0x69, 0x14, 0x63, 0x55, 0x21, 0x0C, 0x7D ]]


In [5]:
# These are translations into python of the code at https://github.com/francisrstokes/AES-C/blob/main/src/aes.c

def MixColumns(state):
    GF_Mult = GaloisMult
    assert type(state) is list and len(state) == 4 and all( len(row) == 4 for row in state )
    temp = [0, 0, 0, 0]
    
    for i in range(4):
        temp[0] = GF_Mult(0x02, state[i][0]) ^ GF_Mult(0x03, state[i][1]) ^ state[i][2] ^ state[i][3]
        temp[1] = state[i][0] ^ GF_Mult(0x02, state[i][1]) ^ GF_Mult(0x03, state[i][2]) ^ state[i][3]
        temp[2] = state[i][0] ^ state[i][1] ^ GF_Mult(0x02, state[i][2]) ^ GF_Mult(0x03, state[i][3])
        temp[3] = GF_Mult(0x03, state[i][0]) ^ state[i][1] ^ state[i][2] ^ GF_Mult(0x02, state[i][3])
        
        state[i][:] = temp
        
def InvMixColumns(state):
    GF_Mult = GaloisMult
    assert type(state) is list and len(state) == 4 and all( len(row) == 4 for row in state )
    temp = [0, 0, 0, 0]
    
    for i in range(4):
        temp[0] = GF_Mult(0x0e, state[i][0]) ^ GF_Mult(0x0b, state[i][1]) ^ GF_Mult(0x0d, state[i][2]) ^ GF_Mult(0x09, state[i][3])
        temp[1] = GF_Mult(0x09, state[i][0]) ^ GF_Mult(0x0e, state[i][1]) ^ GF_Mult(0x0b, state[i][2]) ^ GF_Mult(0x0d, state[i][3])
        temp[2] = GF_Mult(0x0d, state[i][0]) ^ GF_Mult(0x09, state[i][1]) ^ GF_Mult(0x0e, state[i][2]) ^ GF_Mult(0x0b, state[i][3])
        temp[3] = GF_Mult(0x0b, state[i][0]) ^ GF_Mult(0x0d, state[i][1]) ^ GF_Mult(0x09, state[i][2]) ^ GF_Mult(0x0e, state[i][3])

        state[i][:] = temp

In [6]:
def SubstituteWord(word, Sbox = Sbox):
    assert type(word) is int and 0 <= word <= 0xFFFFFFFF
    byte1 = word >> (8*3) & 0xFF
    byte2 = word >> (8*2) & 0xFF
    byte3 = word >> (8*1) & 0xFF
    byte4 = word & 0xFF
    assert all( 0 <= byte <= 0xFF for byte in [ byte1, byte2, byte3, byte4 ] )
    
    top_nibble = byte1 >> 4 & 0xF
    bottom_nibble = byte1 & 0xF
    assert { top_nibble, bottom_nibble } < {*range(16)}
    byte1 = Sbox[top_nibble][bottom_nibble]
    assert 0 <= byte1 <= 0xFF
    
    top_nibble = byte2 >> 4 & 0xF
    bottom_nibble = byte2 & 0xF
    assert { top_nibble, bottom_nibble } < {*range(16)}
    byte2 = Sbox[top_nibble][bottom_nibble]
    assert 0 <= byte2 <= 0xFF
    
    top_nibble = byte3 >> 4 & 0xF
    bottom_nibble = byte3 & 0xF
    assert { top_nibble, bottom_nibble } < {*range(16)}
    byte3 = Sbox[top_nibble][bottom_nibble]
    assert 0 <= byte3 <= 0xFF
    
    top_nibble = byte4 >> 4 & 0xF
    bottom_nibble = byte4 & 0xF
    assert { top_nibble, bottom_nibble } < {*range(16)}
    byte4 = Sbox[top_nibble][bottom_nibble]
    assert 0 <= byte4 <= 0xFF
    
    return ( byte1 << (8*3) ) + ( byte2 << (8*2) ) + ( byte3 << (8*1) ) + byte4

def RotateWord(word):
    assert type(word) is int and word in range(0xFFFFFFFF)
    top_byte = word >> (8*3) & 0xFF
    assert 0 <= top_byte <= 0xFF
    word <<= 8
    word &= 0xFFFFFF00
    word += top_byte
    assert word in range(0xFFFFFFFF)
    return word

def BytesToInt(string: bytes):
    assert type(string) is bytes
    z = 0
    for n in reversed(range(len(string))):
        z += string[n] << (n*8)
    return z

In [7]:
def AesKeyExpansion(key: bytes):
    assert type(key) is bytes and len(key)*8 == 128
    
    # define round constants
    C = 0x01000000, 0x02000000, 0x04000000, 0x08000000, 0x10000000, 0x20000000, 0x40000000, 0x80000000, 0x1B000000, 0x36000000
    assert len(C) == 10
    
    N = 4 # number of 32bit words in key
    R = 11 # number of round keys
    
    W = [None] * (N*R)
    assert len(key) == 16
    K =  BytesToInt(key[0:4]), BytesToInt(key[4:8]), BytesToInt(key[8:12]), BytesToInt(key[12:16])
    assert all( 0 <= word <= 0xFFFFFFFF for word in K )
    
    for w in range(len(W)):
        if w < N:
            W[w] = K[w]
        elif w >= N and w%N == 0:
            W[w] = W[w-N] ^ SubstituteWord(RotateWord(W[w-1])) ^ C[w//N - 1]
        elif w >= N and N > 6 and w%N == 4:
            assert False
            pass
        else:
            W[w] = W[w-N] ^ W[w-1]
            
    assert len(W) == N*R
    assert all( item in range(0xFFFFFFFF) for item in W )
    
    return W
    
    
def RowShiftLeft(row: list, nshifts = 0):
    assert type(row) == list and len(row) == 4
    while nshifts != 0:
        if nshifts > 0:
            row[:] = row[1:] + [ row[0] ]
            nshifts -= 1
        if nshifts < 0:
            row[:] = [ row[-1] ] + row[0:-1]
            nshifts += 1
    assert len(row) == 4
    return None

def Chunkerize(x, chunksize):
    x = list(x)
    assert len(x) % chunksize == 0
    for n in range( len(x) // chunksize ):
        yield x[ n*chunksize : (n+1)*chunksize ]

In [8]:
def RijndaelEncrypt(message: bytes, key: bytes):
    assert type(key) is bytes and len(key) == 16
    
    pad = 16 - len(message) % 16
    message += bytes([pad]) * ( pad if pad < 16 else 0 )
    assert len(message) % 16 == 0
    
    if len(message) > 16:
        return b''.join( RijndaelEncrypt(block, key) for block in Chunkerize(message, 16) )
    
    
    matrix = [ [ *row ] for row in Chunkerize(message, 4) ]
    assert len(matrix) == 4
    assert all( len(row) == 4 for row in matrix )
    
    W = AesKeyExpansion(key) # expanded keys
    assert len(W) == 4 * 11
    
    ## AddRoundKey - the bytes of the first round key are XOR'd with the bytes of the state.
    
    round_keys = W[:4]
    for row, key in zip(matrix, round_keys):
        assert 0 <= key <= 0xFFFFFFFF
        row[0] ^= key >> (8*3) & 0xFF
        row[1] ^= key >> (8*2) & 0xFF
        row[2] ^= key >> (8*1) & 0xFF
        row[3] ^= key >> (8*0) & 0xFF
        
    
    for round_number, round_keys in enumerate(Chunkerize(W[4:], 4)):
        assert len(round_keys) == 4
        assert all( 0 <= key <= 0xFFFFFFFF for key in round_keys )
        None
        assert all( all( 0 <= val <= 0xFF for val in row ) for row in matrix )
        
        # SubBytes - each byte of the state is substituted for a different byte according to a lookup table ("S-box").
        for row in matrix:
            top_nibbles = [ val >> 4 & 0xF for val in row ]
            bottom_nibbles = [ val & 0xF for val in row ]
            row[:] = [ Sbox[top_nibble][bottom_nibble] for top_nibble, bottom_nibble in zip(top_nibbles, bottom_nibbles) ]
        
        # ShiftRows - the last three rows of the state matrix are transposed—shifted over a column or two or three.
        RowShiftLeft(matrix[0], 0)
        RowShiftLeft(matrix[1], 1)
        RowShiftLeft(matrix[2], 2)
        RowShiftLeft(matrix[3], 3)
        
        # MixColumns - matrix multiplication is performed on the columns of the state, combining the four bytes in each column. 
        # This is skipped in the final round.
        if round_number != 9:
            MixColumns(matrix)
        
        for row, key in zip(matrix, round_keys):
            assert 0 <= key <= 0xFFFFFFFF
            row[0] ^= key >> (8*3) & 0xFF
            row[1] ^= key >> (8*2) & 0xFF
            row[2] ^= key >> (8*1) & 0xFF
            row[3] ^= key >> (8*0) & 0xFF
            
    
    assert round_number == 9
    return bytes([ *Unlist(*matrix) ])

In [9]:
message = b"but soft what light through yonder window breaks"
key = b"YELLOW SUBMARINE"

cyphertext = RijndaelEncrypt(message, key)
print(cyphertext)

b'\xc5\x07"\x83\xd5b\x08y\xab\xedI\x8bt\xf4\x12\t\x1b\x93s\xca\x1b[\xb0\x92\xaa\xb9\xd8<$*\xc6\xd2\x0el&\x88\xaf,(q\xc1zo\xdb|!\'\xd2'


In [10]:
def RijndaelDecrypt(cyphertext, key):
    assert type(cyphertext) is bytes 
    assert len(cyphertext) % 16 == 0 
    assert len(cyphertext) > 0
    assert type(key) is bytes and len(key) == 16
    
    if len(cyphertext) > 16:
        return b''.join( RijndaelDecrypt(bytes(block), key) for block in Chunkerize(cyphertext, 16) )
    
    matrix = [ [ *row ] for row in Chunkerize(cyphertext, 4) ]
    assert len(matrix) == 4
    assert all( len(row) == 4 for row in matrix )
    
    W = AesKeyExpansion(key) # expanded keys
    assert len(W) == 4 * 11
        
    for round_number, round_keys in enumerate(reversed([ *Chunkerize(W[4:], 4) ])):
        
        # Key addition
        for row, key in zip(matrix, round_keys):
            assert 0 <= key <= 0xFFFFFFFF
            row[0] ^= key >> (8*3) & 0xFF
            row[1] ^= key >> (8*2) & 0xFF
            row[2] ^= key >> (8*1) & 0xFF
            row[3] ^= key >> (8*0) & 0xFF
            
        if round_number != 0:
            InvMixColumns(matrix)
            
        # ShiftRows - the last three rows of the state matrix are transposed—shifted over a column or two or three.
        RowShiftLeft(matrix[0], 0)
        RowShiftLeft(matrix[1], -1)
        RowShiftLeft(matrix[2], -2)
        RowShiftLeft(matrix[3], -3)
        
        for row in matrix:
            top_nibbles = [ val >> 4 & 0xF for val in row ]
            bottom_nibbles = [ val & 0xF for val in row ]
            row[:] = [ InverseSbox[top_nibble][bottom_nibble] for top_nibble, bottom_nibble in zip(top_nibbles, bottom_nibbles) ]
       
    round_keys = W[:4]
    for row, key in zip(matrix, round_keys):
        assert 0 <= key <= 0xFFFFFFFF
        row[0] ^= key >> (8*3) & 0xFF
        row[1] ^= key >> (8*2) & 0xFF
        row[2] ^= key >> (8*1) & 0xFF
        row[3] ^= key >> (8*0) & 0xFF
        
            
    assert round_number == 9
    
    return bytes([ *Unlist(*matrix) ])


In [11]:
RijndaelDecrypt(cyphertext, key)

b'but soft what light through yonder window breaks'

In [ ]:
# I know that there are some mistakes in here somewhere, because while this correctly encrypts and decrypts 
# messages using just this implementation, it appears to have errors when decrypting cyphertexts produced by
# "official" implementations of AES.  I'm not going to bother tracking down whatever that difference is, because
# I think this serves its purpose as a conceptual implementation of AES which was never intended to be a real implementation.